matplotlib 문제로 그래프 관련된 셀은 모두 markdown셀로 바꾸었습니다!

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8

In [2]:
train_df = pd.read_csv("train.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
#비어 있는 데이터의 개수, %

total = train_df.isnull().sum().sort_values(ascending=False)
percent_1 = train_df.isnull().sum()/train_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis = 1, keys = ['total','%'])
missing_data

,total,%
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2
PassengerId,0,0.0
Survived,0,0.0
Pclass,0,0.0
Name,0,0.0
Sex,0,0.0
SibSp,0,0.0
Parch,0,0.0


In [5]:
train_df.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [6]:
# condition문을 이렇게 작성할 수 있음! 데이터프레임 반환

women = train_df[train_df['Sex'] == 'female']
women

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...,...
880,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S



survived = 'survived'
not_survived = 'not survived'
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10, 4))

# 여자 & 생존 / 여자 & 사망인원수를 체크하기 위해 2중 condition문으로 작성

women = train_df[train_df['Sex']=='female']
men = train_df[train_df['Sex']=='male']
ax = sns.distplot(women[women['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[0], kde =False)
ax = sns.distplot(women[women['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[0], kde =False)
ax.legend()
ax.set_title('Female')
ax = sns.distplot(men[men['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[1], kde = False)
ax = sns.distplot(men[men['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[1], kde = False)
ax.legend()
_ = ax.set_title('Male')



#embarked 기준으로 FacetGrid만 만들면 이렇게 됨

FacetGrid = sns.FacetGrid(train_df, row='Embarked', size=4.5, aspect=1.6)
FacetGrid.add_legend()
FacetGrid



FacetGrid = sns.FacetGrid(train_df, row='Embarked', size=4.5, aspect=1.6)
FacetGrid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette=None,  order=None, hue_order=None )
FacetGrid.add_legend()



sns.barplot(x='Pclass', y='Survived', data=train_df)


# 두 가지 기준으로 FacetGrid 만들수도 있음!

grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()


In [12]:
test_df = pd.read_csv("test.csv")

In [13]:
data = [train_df, test_df]
data

[     PassengerId  Survived  Pclass  \
 0              1         0       3   
 1              2         1       1   
 2              3         1       3   
 3              4         1       1   
 4              5         0       3   
 ..           ...       ...     ...   
 886          887         0       2   
 887          888         1       1   
 888          889         0       3   
 889          890         1       1   
 890          891         0       3   
 
                                                   Name     Sex   Age  SibSp  \
 0                              Braund, Mr. Owen Harris    male  22.0      1   
 1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
 2                               Heikkinen, Miss. Laina  female  26.0      0   
 3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
 4                             Allen, Mr. William Henry    male  35.0      0   
 ..                                               

In [14]:
#데이터프레임에 새로운 열 추가하기 -> df['추가할 열 이름'] = 데이터(배열, 이미 있는 열 등등)

train_df['relatives'] = train_df['SibSp'] + train_df['Parch']
train_df.drop(['relatives'], axis = 1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [15]:
# for dataset in data:
# data라는 리스트에 있는 1번째 데이터프레임, 2번째 데이터프레임에 각각 아래 구문을 실행!
# for i in list와 같음

for dataset in data:
  dataset['relatives'] = dataset['SibSp'] + dataset['Parch']
  dataset.loc[dataset['relatives'] > 0, 'not_alone'] = 0
  dataset.loc[dataset['relatives'] == 0, 'not_alone'] = 1
  dataset['not_alone'] = dataset['not_alone'].astype(int)

train_df['not_alone'].value_counts()

1    537
0    354
Name: not_alone, dtype: int64


axes = sns.factorplot('relatives','Survived', 
                      data=train_df, aspect = 2.5, )


In [17]:
train_df = train_df.drop(['PassengerId'], axis = 1)

In [18]:
#??

import re
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
data = [train_df, test_df]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)
# we can now drop the cabin feature
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

In [19]:
random_list = np.random.randint(1, 100, size = 10)
random_list

array([78, 21, 64, 87, 19, 47, 12, 77, 22, 29])

In [20]:
Pclass_slice = train_df['Pclass'].copy()
Pclass_slice

0      3
1      1
2      3
3      1
4      3
      ..
886    2
887    1
888    3
889    1
890    3
Name: Pclass, Length: 891, dtype: int64

In [21]:
#데이터프레임에서 열 한 개의 값을 리스트의 값으로 바꾸고 싶을 때!
#slice[] 대괄호 내부에는 인덱스 값 넣어야 함!

Pclass_slice[0:10] = random_list
Pclass_slice

0      78
1      21
2      64
3      87
4      19
       ..
886     2
887     1
888     3
889     1
890     3
Name: Pclass, Length: 891, dtype: int64

In [22]:
train_df['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [23]:
common_value = 'S'
data = [train_df, test_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(common_value)

In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Survived   891 non-null    int64  
 1   Pclass     891 non-null    int64  
 2   Name       891 non-null    object 
 3   Sex        891 non-null    object 
 4   Age        714 non-null    float64
 5   SibSp      891 non-null    int64  
 6   Parch      891 non-null    int64  
 7   Ticket     891 non-null    object 
 8   Fare       891 non-null    float64
 9   Embarked   891 non-null    object 
 10  relatives  891 non-null    int64  
 11  not_alone  891 non-null    int32  
 12  Deck       891 non-null    int32  
dtypes: float64(2), int32(2), int64(5), object(4)
memory usage: 83.7+ KB


In [25]:
data = [train_df, test_df]

for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)

In [26]:
data = [train_df, test_df]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    # 데이터프레임 열에 .map(dict형) 사용해서 텍스트 -> 숫자로 변환하기!
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)

train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

In [27]:
genders = {'male':0, 'female':1}
data = [train_df, test_df]

for dataset in data:
  dataset['Sex'] = dataset['Sex'].map(genders)

In [28]:
train_df['Ticket'].describe()

count        891
unique       681
top       347082
freq           7
Name: Ticket, dtype: object

In [29]:
train_df = train_df.drop(['Ticket'], axis=1)
test_df = test_df.drop(['Ticket'], axis=1)

In [30]:
embark = {'S':0, 'C':1, 'Q':2}
data = [train_df, test_df]

for dataset in data:
  dataset['Embarked'] = dataset['Embarked'].map(embark)

train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Deck,Title
0,0,3,0,22.0,1,0,7,0,1,0,8,1
1,1,1,1,38.0,1,0,71,1,1,0,3,3
2,1,3,1,26.0,0,0,7,0,0,1,8,2
3,1,1,1,35.0,1,0,53,0,1,0,3,3
4,0,3,0,35.0,0,0,8,0,0,1,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13,0,0,1,8,5
887,1,1,1,19.0,0,0,30,0,0,1,2,2
888,0,3,1,NaN,1,2,23,0,3,0,8,2
889,1,1,0,26.0,0,0,30,1,0,1,3,1


In [31]:
#Age변수 비어있는 칸 채우기!
#이 블럭에서는 Age제외한 데이터프레임 만듦

from sklearn.neighbors import KNeighborsRegressor
# train
impute_train = train_df[train_df.Age.isnull()].drop(['Age'], axis=1)
impute_train.describe()
# test
impute_test = test_df[test_df.Age.isnull()].drop(['Age'], axis=1)
impute_test.describe()

,PassengerId,Pclass,Sex,SibSp,Parch,Fare,Embarked,relatives,not_alone,Deck,Title
count,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,1102.186047,2.732558,0.290698,0.313953,0.372093,14.511628,0.790698,0.686047,0.779070,7.848837,1.546512
std,113.626997,0.640302,0.456748,0.973310,1.471666,14.171010,0.855656,1.953950,0.417307,0.711543,0.876584
min,902.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000
25%,1004.250000,3.000000,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000,1.000000,8.000000,1.000000
50%,1105.500000,3.000000,0.000000,0.000000,0.000000,7.500000,1.000000,0.000000,1.000000,8.000000,1.000000
75%,1181.750000,3.000000,1.000000,0.000000,0.000000,15.000000,2.000000,0.000000,1.000000,8.000000,2.000000
max,1309.000000,3.000000,1.000000,8.000000,9.000000,69.000000,2.000000,10.000000,1.000000,8.000000,4.000000


In [32]:
#Age를 제외한 나머지 특징들을 이용해서 KNN-Regressor으로 나이를 예측함

# train
knr = KNeighborsRegressor()
titanic_knn_train = train_df[train_df.Age.notnull()]
X_train = titanic_knn_train.drop(['Age'], axis = 1)
y_train = titanic_knn_train.Age
knr.fit(X_train, y_train)
imputed_train_ages = knr.predict(impute_train)
imputed_train_ages
# test
knr = KNeighborsRegressor()
titanic_knn_test = test_df[test_df.Age.notnull()]
X_test = titanic_knn_test.drop(['Age'], axis = 1)
y_test = titanic_knn_test.Age
X_test.isnull().sum()
knr.fit(X_test, y_test)
imputed_test_ages = knr.predict(impute_test)
imputed_test_ages

array([22.   , 40.4  , 39.6  , 39.6  , 28.9  , 28.4  , 34.4  , 31.6  ,
       31.   , 27.2  , 33.3  , 25.6  , 21.   , 24.2  , 20.6  , 23.2  ,
       24.2  , 25.   , 23.2  , 20.   , 20.   , 23.   , 19.5  , 20.9  ,
       29.6  , 30.6  , 33.8  , 29.9  , 29.2  , 34.6  , 33.1  , 28.   ,
       27.2  , 29.2  , 27.8  , 28.6  , 37.6  , 38.   , 22.2  , 19.866,
       25.466, 26.666, 29.4  , 23.8  , 23.4  , 29.9  , 21.6  , 24.   ,
       20.4  , 30.384, 23.5  , 25.1  , 25.3  , 18.   , 23.4  , 18.   ,
       19.4  , 24.   , 19.55 , 18.15 , 21.95 , 29.4  , 30.2  , 30.2  ,
       18.6  , 31.4  , 38.   , 34.6  , 20.766, 22.6  , 28.4  , 26.4  ,
       34.5  , 24.4  , 28.8  , 28.8  , 30.3  , 30.2  , 22.8  , 29.4  ,
       29.4  , 22.5  , 22.5  , 22.5  , 22.5  , 26.5  ])

In [33]:
imputed_train_ages

array([39.8  , 29.4  , 18.8  , 27.4  , 18.8  , 33.2  , 28.784, 18.8  ,
       24.7  , 27.4  , 22.6  , 27.9  , 18.8  , 26.   , 50.4  , 50.2  ,
       14.734, 33.2  , 22.6  , 18.8  , 22.6  , 22.6  , 33.2  , 31.   ,
       22.   , 22.6  , 44.4  , 19.   , 23.2  , 33.2  , 22.6  , 29.8  ,
       34.8  , 52.6  , 15.4  , 29.8  , 27.7  , 36.2  , 26.1  , 44.4  ,
       18.8  , 29.8  , 39.3  , 33.2  , 17.8  , 28.6  , 26.5  , 17.7  ,
       33.2  , 32.6  , 44.4  , 20.9  , 21.9  , 18.8  , 33.4  , 44.7  ,
       50.2  , 37.4  , 18.8  , 24.4  , 30.3  , 22.6  , 28.   , 29.8  ,
       23.2  , 37.6  , 33.2  , 31.8  , 50.4  , 27.4  , 18.8  , 18.8  ,
       27.9  , 18.8  , 18.8  , 30.   , 33.2  , 44.4  , 17.8  , 33.2  ,
       42.   , 33.4  , 26.   , 27.4  , 33.2  , 44.4  , 31.8  , 25.   ,
       21.6  , 22.6  , 42.   , 44.4  , 22.6  , 33.4  , 44.4  , 33.2  ,
       38.8  , 33.4  , 17.8  , 21.6  , 24.6  , 27.7  , 20.9  , 52.2  ,
       22.6  , 35.6  , 27.4  , 27.4  , 28.4  , 27.4  , 20.2  , 31.8  ,
      

In [34]:
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Deck,Title
0,0,3,0,22.0,1,0,7,0,1,0,8,1
1,1,1,1,38.0,1,0,71,1,1,0,3,3
2,1,3,1,26.0,0,0,7,0,0,1,8,2
3,1,1,1,35.0,1,0,53,0,1,0,3,3
4,0,3,0,35.0,0,0,8,0,0,1,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13,0,0,1,8,5
887,1,1,1,19.0,0,0,30,0,0,1,2,2
888,0,3,1,NaN,1,2,23,0,3,0,8,2
889,1,1,0,26.0,0,0,30,1,0,1,3,1


In [35]:
#Age중에서 nan인 인덱스에만 우리가 예측한 나이 값 리스트를 대입함!

age_slice = train_df['Age'].copy()
age_slice[np.isnan(age_slice)] = imputed_train_ages
train_df['Age'] = age_slice
train_df['Age'] = train_df['Age'].astype(int)

age_slice2 = test_df['Age'].copy()
age_slice2[np.isnan(age_slice2)] = imputed_test_ages
test_df['Age'] = age_slice2
test_df['Age'] = test_df['Age'].astype(int)


train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Deck,Title
0,0,3,0,22,1,0,7,0,1,0,8,1
1,1,1,1,38,1,0,71,1,1,0,3,3
2,1,3,1,26,0,0,7,0,0,1,8,2
3,1,1,1,35,1,0,53,0,1,0,3,3
4,0,3,0,35,0,0,8,0,0,1,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27,0,0,13,0,0,1,8,5
887,1,1,1,19,0,0,30,0,0,1,2,2
888,0,3,1,20,1,2,23,0,3,0,8,2
889,1,1,0,26,0,0,30,1,0,1,3,1


In [36]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Survived   891 non-null    int64
 1   Pclass     891 non-null    int64
 2   Sex        891 non-null    int64
 3   Age        891 non-null    int32
 4   SibSp      891 non-null    int64
 5   Parch      891 non-null    int64
 6   Fare       891 non-null    int32
 7   Embarked   891 non-null    int64
 8   relatives  891 non-null    int64
 9   not_alone  891 non-null    int32
 10  Deck       891 non-null    int32
 11  Title      891 non-null    int64
dtypes: int32(4), int64(8)
memory usage: 69.7 KB


In [37]:
#Categorization

data = [train_df, test_df]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

# let's see how it's distributed 
train_df['Age'].value_counts()

4    182
6    177
3    135
2    120
5    113
1     96
0     68
Name: Age, dtype: int64

In [38]:
#pandas qcut함수
#6개의 범위로 연속형 데이터를 나눠줌. 각 범위에 들어가는 데이터의 수는 대강 비슷하게 맞춰줌!

pd.qcut(train_df['Fare'], 6)

0      (-0.001, 7.0]
1      (52.0, 512.0]
2      (-0.001, 7.0]
3      (52.0, 512.0]
4         (7.0, 8.0]
           ...      
886      (8.0, 14.0]
887     (26.0, 52.0]
888     (14.0, 26.0]
889     (26.0, 52.0]
890    (-0.001, 7.0]
Name: Fare, Length: 891, dtype: category
Categories (6, interval[float64, right]): [(-0.001, 7.0] < (7.0, 8.0] < (8.0, 14.0] < (14.0, 26.0] < (26.0, 52.0] < (52.0, 512.0]]

In [39]:
data = [train_df, test_df]

for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7.00, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.00) & (dataset['Fare'] <= 8), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 8) & (dataset['Fare'] <= 14), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 14) & (dataset['Fare'] <= 26), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 26) & (dataset['Fare'] <= 52), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 52, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)
  
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Deck,Title
0,0,3,0,2,1,0,0,0,1,0,8,1
1,1,1,1,5,1,0,5,1,1,0,3,3
2,1,3,1,3,0,0,0,0,0,1,8,2
3,1,1,1,5,1,0,5,0,1,0,3,3
4,0,3,0,5,0,0,1,0,0,1,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,3,0,0,2,0,0,1,8,5
887,1,1,1,2,0,0,4,0,0,1,2,2
888,0,3,1,2,1,2,3,0,3,0,8,2
889,1,1,0,3,0,0,4,1,0,1,3,1


In [40]:
train_df.groupby(['Fare', 'Survived']).size()

Fare  Survived
0     0           189
      1            52
1     0            60
      1            10
2     0            94
      1            52
3     0            85
      1            80
4     0            76
      1            47
5     0            45
      1           101
dtype: int64

In [41]:
#새로운 feature 만들기!

data = [train_df, test_df]
for dataset in data:
    dataset['Age_Class']= dataset['Age']* dataset['Pclass']

In [42]:
for dataset in data:
    dataset['Fare_Per_Person'] = dataset['Fare']/(dataset['relatives']+1)
    dataset['Fare_Per_Person'] = dataset['Fare_Per_Person'].astype(int)
# Let's take a last look at the training set, before we start training the models.
train_df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Deck,Title,Age_Class,Fare_Per_Person
0,0,3,0,2,1,0,0,0,1,0,8,1,6,0
1,1,1,1,5,1,0,5,1,1,0,3,3,5,2
2,1,3,1,3,0,0,0,0,0,1,8,2,9,0
3,1,1,1,5,1,0,5,0,1,0,3,3,5,2
4,0,3,0,5,0,0,1,0,0,1,8,1,15,1
5,0,3,0,5,0,0,1,2,0,1,8,1,15,1
6,0,1,0,6,0,0,4,0,0,1,5,1,6,4
7,0,3,0,0,3,1,3,0,4,0,8,4,0,0
8,1,3,1,3,0,2,2,0,2,0,8,3,9,0
9,1,2,1,1,1,0,4,1,1,0,8,3,2,2


In [43]:
test_df.head(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Deck,Title,Age_Class,Fare_Per_Person
0,892,3,0,5,0,0,0,2,0,1,8,1,15,0
1,893,3,1,6,1,0,0,0,1,0,8,3,18,0
2,894,2,0,6,0,0,2,2,0,1,8,1,12,2
3,895,3,0,3,0,0,1,0,0,1,8,1,9,1
4,896,3,1,2,1,1,2,0,2,0,8,3,6,0
5,897,3,0,1,0,0,2,0,0,1,8,1,3,2
6,898,3,1,4,0,0,0,2,0,1,8,2,12,0
7,899,2,0,3,1,1,4,0,2,0,8,1,6,1
8,900,3,1,1,0,0,0,1,0,1,8,3,3,0
9,901,3,0,2,2,0,3,0,2,0,8,1,6,1


In [44]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

In [45]:
#Stochastic Gradient Descent

sgd = linear_model.SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)

sgd.score(X_train, Y_train)

acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)

In [46]:
#Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

In [47]:
#Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

acc_log = round(logreg.score(X_train, Y_train) * 100, 2)

C:\Users\LG\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [48]:
# K Nearest Neighbor

knn = KNeighborsClassifier(n_neighbors = 3) 
knn.fit(X_train, Y_train)  
Y_pred = knn.predict(X_test)  
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)

In [49]:
#Gaussian Naive Bayes

gaussian = GaussianNB() 
gaussian.fit(X_train, Y_train)  
Y_pred = gaussian.predict(X_test)  
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)

In [50]:
#Perceptron

perceptron = Perceptron(max_iter=5)
perceptron.fit(X_train, Y_train)

Y_pred = perceptron.predict(X_test)

acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)

C:\Users\LG\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


In [51]:
#Linear Support Vector Machine

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)

Y_pred = linear_svc.predict(X_test)

acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)

C:\Users\LG\anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [52]:
#Decision Tree

decision_tree = DecisionTreeClassifier() 
decision_tree.fit(X_train, Y_train)  
Y_pred = decision_tree.predict(X_test)  
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)

In [53]:
results = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [acc_linear_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_decision_tree]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.reset_index(drop = True)
result_df.head(9)

,Model,Score
0,Random Forest,93.04
1,Decision Tree,93.04
2,KNN,85.97
3,Logistic Regression,82.38
4,Support Vector Machines,81.26
5,Stochastic Gradient Decent,79.80
6,Naive Bayes,78.68
7,Perceptron,77.67


In [54]:
#K-Fold Cross Validation

from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(rf, X_train, Y_train, cv=10, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.75555556 0.82022472 0.76404494 0.83146067 0.87640449 0.86516854
 0.83146067 0.82022472 0.87640449 0.82022472]
Mean: 0.8261173533083644
Standard Deviation: 0.03946620216711727


In [55]:
decf = DecisionTreeClassifier()
scores = cross_val_score(decf, X_train, Y_train, cv = 10, scoring = 'accuracy')
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.73333333 0.80898876 0.75280899 0.82022472 0.86516854 0.78651685
 0.80898876 0.79775281 0.84269663 0.79775281]
Mean: 0.8014232209737827
Standard Deviation: 0.03672009130457036


In [56]:
knnf = KNeighborsClassifier(n_neighbors = 5) 
scores = cross_val_score(knnf, X_train, Y_train, cv = 10, scoring = 'accuracy')
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.76666667 0.83146067 0.76404494 0.82022472 0.80898876 0.85393258
 0.80898876 0.82022472 0.83146067 0.83146067]
Mean: 0.81374531835206
Standard Deviation: 0.027151904896079666


In [57]:
importances = pd.DataFrame({
    'feature':X_train.columns,
    'importance':np.round(random_forest.feature_importances_,3)})

importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances.head(15)

,importance
feature,
Title,0.200
Sex,0.164
Age_Class,0.101
Deck,0.085
Age,0.079
Fare,0.072
Pclass,0.065
Fare_Per_Person,0.056
Embarked,0.051



importances.plot.bar()


Removing Parch and not_alone

In [59]:
train_df  = train_df.drop("not_alone", axis=1)
test_df  = test_df.drop("not_alone", axis=1)

train_df  = train_df.drop("Parch", axis=1)
test_df  = test_df.drop("Parch", axis=1)

In [60]:
#oob score를 이용해 정확도 측정하기

random_forest = RandomForestClassifier(n_estimators=100, oob_score = True)
random_forest.fit(X_train, Y_train)
Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
print(round(acc_random_forest,2,), "%")

93.04 %


C:\Users\LG\anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [61]:
print("oob score:", round(random_forest.oob_score_, 4)*100, "%")

oob score: 81.93 %


#Hyperparameter Tuning
#분류/회귀 알고리즘에 사용되는 하이퍼파라미터들을 순차적으로 입력해 학습한 후,
#그중 가장 좋은 파라미터를 알려줌!


param_grid = { "criterion" : ["gini", "entropy"], 
              "min_samples_leaf" : [1, 5, 10, 25, 50, 70], 
              "min_samples_split" : [2, 4, 10, 12, 16, 18, 25, 35], 
              "n_estimators": [100, 400, 700, 1000, 1500]}

from sklearn.model_selection import GridSearchCV, cross_val_score

rf = RandomForestClassifier(n_estimators=100, max_features='auto', 
                            oob_score=True, random_state=1, n_jobs=-1)

clf = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1)
clf.fit(X_train, Y_train)


print('best parameters : ', clf.best_params_)



In [63]:
#앞에서 찾은 좋은 hyperparameter 대입해서 다시 학습

random_forest = RandomForestClassifier(criterion = 'gini',
                                       min_samples_leaf = 1,
                                       min_samples_split = 10,
                                       n_estimators = 700,
                                       max_features = 'auto',
                                       oob_score = True,
                                       random_state = 1,
                                       n_jobs = -1)

random_forest.fit(X_train, Y_train)
Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

print("oob score : ", round(random_forest.oob_score_, 4) * 100, "%")

C:\Users\LG\anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


oob score :  83.61 %


In [64]:
#Confusion Matrix

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

predictions = cross_val_predict(random_forest, X_train, Y_train, cv=3)
confusion_matrix(Y_train, predictions)

C:\Users\LG\anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\LG\anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\LG\anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


array([[498,  51],
       [ 98, 244]], dtype=int64)

1행 = 죽은 사람들 중에서, 0으로 성공적으로 예측한 인원수 500명, 1으로 잘못 예측한 인원수 49명

2행 = 산 사람들 중에서, 0으로 잘못 예측한 인원 98명, 1으로 성공적으로 예측한 인원수 244명

In [65]:
from sklearn.metrics import precision_score, recall_score

print("Precision:", precision_score(Y_train, predictions))
print("Recall:",recall_score(Y_train, predictions))

Precision: 0.8271186440677966
Recall: 0.7134502923976608


Precision = 성공적으로 예측한 사람 퍼센트
Recall = 산 사람 중에 71%를 정확히 예측함

In [66]:
from sklearn.metrics import f1_score
f1_score(Y_train, predictions)

0.7660910518053375

Precision & Recall 합친 F-score

#precision-recall curve


from sklearn.metrics import precision_recall_curve

# getting the probabilities of our predictions
y_scores = random_forest.predict_proba(X_train)
y_scores = y_scores[:,1]

precision, recall, threshold = precision_recall_curve(Y_train, y_scores)
def plot_precision_and_recall(precision, recall, threshold):
    plt.plot(threshold, precision[:-1], "r-", label="precision", linewidth=5)
    plt.plot(threshold, recall[:-1], "b", label="recall", linewidth=5)
    plt.xlabel("threshold", fontsize=19)
    plt.legend(loc="upper right", fontsize=19)
    plt.ylim([0, 1])

plt.figure(figsize=(14, 7))
plot_precision_and_recall(precision, recall, threshold)
plt.show()




def plot_precision_vs_recall(precision, recall):
    plt.plot(recall, precision, "g--", linewidth=2.5)
    plt.ylabel("recall", fontsize=19)
    plt.xlabel("precision", fontsize=19)
    plt.axis([0, 1.5, 0, 1.5])

plt.figure(figsize=(14, 7))
plot_precision_vs_recall(precision, recall)
plt.show()


#ROC AUC Curve
#recall(true positive; 성공적으로 1으로 예측) / false positive; 잘못 1으로 예측 그래프

from sklearn.metrics import roc_curve
# compute true positive rate and false positive rate
false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_train, y_scores)
# plotting them against each other
def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)

plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)
plt.show()


from sklearn.metrics import roc_auc_score
r_a_score = roc_auc_score(Y_train, y_scores)
print("ROC-AUC-Score:", r_a_score)


In [71]:
Y_prediction

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [72]:
output = pd.DataFrame({'PassengerID': test_df['PassengerId'].values.tolist(), 'Survived': Y_prediction})
output.to_csv('submission.csv', index = False)